In [ ]:
import pandas as pd
import pymc3 as pm
import numpy as np
import seaborn as sns
import arviz as az
import matplotlib.pyplot as plt
import spc_os
from spc_vis import my_plot_ppc

RANDOM_SEED = 28101990
import pickle

from math import ceil
import theano.tensor as T
import scipy.stats as st

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black
%load_ext watermark
%watermark -n -u -v -iv -w -m

In [ ]:
raw_data_dir = "../data/raw/"
interim_data_dir = "../data/interim/"
processed_data_dir = "../data/processed/"
external_data_dir = "../data/external/"
models_dir = "../models/"

In [ ]:
model_path = models_dir + f"tpore_survival_analysis_same_membrane.nc"

In [ ]:
infer = True
save_data = True
print(model_path)

# Load data

In [ ]:
df = pd.read_csv(f"{processed_data_dir}data.csv").drop("Unnamed: 0", axis=1)

In [ ]:
df.Replica = df.membrane

In [ ]:
df.Replica = df.Replica.astype("category")

In [ ]:
df["Replica_enc"] = df.Replica.cat.codes

In [ ]:
category_dic = {i: cat for i, cat in enumerate(np.unique(df["Replica"]))}

In [ ]:
category_dic

In [ ]:
n_categories = len(category_dic)

In [ ]:
dummies = pd.get_dummies(df.Replica, prefix="Replica")

In [ ]:
for col in dummies.columns:
    df[col] = dummies[col]

In [ ]:
df.tpore = df.tpore * 10
df.tpore = df.tpore.astype(int)

In [ ]:
df.head()

## Visualize Data

In [ ]:
df["tpore"].groupby(df["Replica"]).describe()

In [ ]:
_ = df["tpore"].hist(by=df["Replica"], sharex=True, density=True, bins=10)

In [ ]:
_ = df["tpore"].hist(bins=50)

## Visualize Priors

These are the shapes of the priors used.

In [ ]:
beta = 1
alpha = 1
d = st.gamma(scale=1 / beta, a=alpha)
x = np.linspace(0, 10, 100)
tau_0_pdf = d.pdf(x)
plt.plot(x, tau_0_pdf, "k-", lw=2)
plt.xlabel("lambda0(t)")

## Prepare data

In [ ]:
n_sims = df.shape[0]
sims = np.arange(n_sims)
interval_length = 15  # 1.5 ns
interval_bounds = np.arange(0, df.tpore.max() + interval_length + 1, interval_length)
n_intervals = interval_bounds.size - 1
intervals = np.arange(n_intervals)

In [ ]:
last_period = np.floor((df.tpore - 0.01) / interval_length).astype(int)

pore = np.zeros((n_sims, n_intervals))
pore[sims, last_period] = np.ones(n_sims)

In [ ]:
exposure = (
    np.greater_equal.outer(df.tpore.values, interval_bounds[:-1]) * interval_length
)
exposure[sims, last_period] = df.tpore - interval_bounds[last_period]

## Run Model

In [ ]:
with pm.Model() as model:

    lambda0 = pm.Gamma("lambda0", 5, 1, shape=n_intervals)

    beta = pm.Normal("beta", 0, sigma=100, shape=(n_categories))

    lambda_ = pm.Deterministic(
        "lambda_", T.outer(T.exp(T.dot(beta, dummies.T)), lambda0)
    )
    mu = pm.Deterministic("mu", exposure * lambda_)
    exp_beta = pm.Deterministic("exp_beta", np.exp(beta))

    obs = pm.Poisson(
        "obs",
        mu,
        observed=pore,
    )

In [ ]:
pm.model_to_graphviz(model)

In [ ]:
%%time
if infer:
    with model:
        trace = pm.sample(1000, tune=1000, random_seed=RANDOM_SEED, return_inferencedata=True, cores=8)
else:
     trace=az.from_netcdf(model_path)

In [ ]:
if infer:
    trace.posterior = trace.posterior.reset_index(
        ["beta_dim_0", "exp_beta_dim_0", "lambda0_dim_0"], drop=True
    )
    trace = trace.rename(
        {
            "lambda0_dim_0": "t",
            "beta_dim_0": "Membrane",
            "exp_beta_dim_0": "Membrane",
        }
    )
    trace = trace.assign_coords(
        t=interval_bounds[:-1] / 10,
        Membrane=list(category_dic.values()),
    )

## Posterior and prior predictive and 

In [ ]:
%%time
if infer:
    with model:
        ppc = pm.sample_posterior_predictive(trace,  random_seed=RANDOM_SEED, samples=1000)
        print('Done infering.')
    trace = az.concat(trace, az.from_pymc3(posterior_predictive=ppc))

In [ ]:
%%time
# Prior predictive not important: uninformative priors
#if infer:
#    with model:
#        prior = pm.sample_prior_predictive(random_seed=RANDOM_SEED, samples=1000)
#        trace.extend(az.from_pymc3(prior=prior))

## Convergences

In [ ]:
with az.rc_context(rc={"plot.max_subplots": None}):
    az.plot_trace(trace, var_names=["beta", "lambda0"])

In [ ]:
with az.rc_context(rc={"plot.max_subplots": None}):
    az.plot_autocorr(trace, combined=True, var_names=["lambda0", "beta"])

In [ ]:
def get_hdi(x, axis, alpha=0.06):
    x_mean = np.nanmedian(x, axis=axis)
    percentiles = 100 * np.array([alpha / 2.0, 1.0 - alpha / 2.0])
    hdi = np.nanpercentile(x, percentiles, axis=axis)

    return x_mean, hdi

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
freq0, bins = np.histogram(
    np.where(pore[:, :] == 1)[1], density=True, bins=np.arange(n_intervals)
)
n_draws = trace.posterior_predictive.obs.values.shape[1]
l = []
for draw in range(n_draws):
    freq, _ = np.histogram(
        np.where(trace.posterior_predictive.obs.values[0, draw, :, :])[1],
        bins=bins,
        density=True,
    )
    l.append(freq)
l = np.array(l)
y, hdi = get_hdi(l, 0)
ax.errorbar(
    bins[:-1] * interval_length / 10,
    y / interval_length * 10,
    yerr=hdi / interval_length * 10,
    label="posterior predictive",
    lw=2,
    capsize=6,
)
ax.bar(
    bins[:-1] * interval_length / 10,
    freq0 / interval_length * 10,
    color="C1",
    width=interval_length / 10,
    label="observed binned",
)
sns.kdeplot(
    df.tpore / 10,
    ax=ax,
    common_norm=True,
    # bw_adjust=0.3,
    color="grey",
    label="data",
    lw=2.5
    # gridsize=10,
)
ax.set_xlabel("t-pore (ns)")
ax.set_ylabel("p(t-pore)")
ax.legend()
fig.tight_layout()

## Analyze

### Plot posterior

In [ ]:
variable = "lambda0"
ax = az.plot_forest(trace, var_names=variable, combined=True)
ax[0].set_xlabel("lambda0[t]")

In [ ]:
variable = "beta"
ax = az.plot_forest(trace, var_names=variable, combined=True)
ax[0].set_xlabel("beta")

In [ ]:
variable = "exp_beta"
ax = az.plot_forest(trace, var_names=variable, combined=True)
ax[0].set_xlabel("exp(beta)")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 7))
lambda0 = trace.posterior.lambda0.values
beta = trace.posterior.beta.values
y, hdi = get_hdi(lambda0, (0, 1))
x = interval_bounds[:-1] * 10
ax[0].fill_between(x, hdi[0], hdi[1], alpha=0.25, step="pre", color="grey")
ax[0].step(x, y, label="baseline", color="grey")
for i in range(n_categories):
    lam = np.exp(beta[:, :, [i]]) * lambda0
    y, hdi = get_hdi(lam, (0, 1))
    ax[1].fill_between(x, hdi[0], hdi[1], alpha=0.25, step="pre")
    ax[1].step(x, y, label=f"{category_dic[i]}")

for a in ax:
    a.legend(loc="best")
    a.set_ylabel("p(lambda0)")
    a.set_xlabel("lambda (ns)^-1")

To DO
+ more categories
+ t-dependent B

## Model?

In [ ]:
print(model_path)

In [ ]:
if save_data:
    spc_os.remove(model_path)
    trace.to_netcdf(model_path)